In [64]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import time
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from IPython.display import display
import time

import gc

In [39]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
pd.set_option("display.max_columns", 1000)


In [88]:
log = pd.read_csv("~/environment/log.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [4]:
print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [5]:
log.loc[:,"user_id_str"] = log["user_id"].apply(lambda x:str(x))
log.loc[:,"event_id_str"] = log["event_id"].apply(lambda x:str(x))
log.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128


In [6]:
data1 = pd.merge(log,events, on="event_id")
data = pd.merge(data1, users, on="user_id")
data = data.rename(columns = {"prefecture_x": "event_prefecture", "prefecture_y":"user_prefecture"})
data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


In [7]:
data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10


In [8]:
data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [9]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]



In [10]:
data = data[features + target]
#naの行消す
data = data.dropna()
data = data.reset_index(drop=True)
print(data.shape)
data.head()

(1442610, 18)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month,action_type
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6,3
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1


In [11]:
X = data[features]
print(X.shape)
X.head()

(1442610, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6


In [75]:
def prefecture_labeling(x):
     #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["prefecture"]
    df.columns = column_name
    
    return df
    

def gender_labeling(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["user_gender"]
    df.columns = column_name
    
    return df
    
    

In [13]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

(1442610, 1)


,event_prefecture
0,27
1,8
2,8
3,8
4,8


In [14]:
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

(1442610, 1)


,user_prefecture
0,43
1,43
2,43
3,43
4,43


In [15]:
gen = gender_labeling(X["gender"])
print(gen.shape)
gen.head()

(1442610, 1)


,gender
0,0
1,0
2,0
3,0
4,0


In [16]:
temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()


(1442610, 14)
(1442610, 17)


,event_year,event_month,age,male_age_lower,male_age_upper,user_year,event_hour,event_weekday,female_price,female_age_upper,male_price,user_month,event_day,female_age_lower,gender,event_prefecture,user_prefecture
0,2015,3,34,27,39.0,2012,14,6,4000.0,39.0,6500.0,6,22,27,0,27,43
1,2017,6,34,26,38.0,2012,10,6,2000.0,36.0,6000.0,6,11,24,0,8,43
2,2017,6,34,26,38.0,2012,10,6,2000.0,36.0,6000.0,6,11,24,0,8,43
3,2017,6,34,26,38.0,2012,10,6,2000.0,36.0,6000.0,6,11,24,0,8,43
4,2017,6,34,26,38.0,2012,10,6,2000.0,36.0,6000.0,6,11,24,0,8,43


In [17]:
X = X.astype(np.int64)

In [18]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1442610, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


# 学習

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1154088,)
float64


In [21]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(288522,)
float64


In [22]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = categorical)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature = categorical)

In [23]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [24]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's l2: 0.55134
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.518201
[3]	valid_0's l2: 0.491125
[4]	valid_0's l2: 0.469136
[5]	valid_0's l2: 0.451271
[6]	valid_0's l2: 0.43672
[7]	valid_0's l2: 0.424873
[8]	valid_0's l2: 0.415222
[9]	valid_0's l2: 0.40729
[10]	valid_0's l2: 0.40082
[11]	valid_0's l2: 0.39547
[12]	valid_0's l2: 0.39108
[13]	valid_0's l2: 0.387395
[14]	valid_0's l2: 0.384436
[15]	valid_0's l2: 0.381942
[16]	valid_0's l2: 0.379821
[17]	valid_0's l2: 0.378078
[18]	valid_0's l2: 0.376565
[19]	valid_0's l2: 0.375346
[20]	valid_0's l2: 0.374261
[21]	valid_0's l2: 0.373267
[22]	valid_0's l2: 0.372424
[23]	valid_0's l2: 0.371752
[24]	valid_0's l2: 0.371078
[25]	valid_0's l2: 0.370513
[26]	valid_0's l2: 0.369995
[27]	valid_0's l2: 0.369551
[28]	valid_0's l2: 0.369137
[29]	valid_0's l2: 0.368748
[30]	valid_0's l2: 0.368414
[31]	valid_0's l2: 0.368133
[32]	valid_0's l2: 0.367735
[33]	valid_0's l2: 0.367468
[34]	valid_0's l2: 0.367118
[

[287]	valid_0's l2: 0.352088
[288]	valid_0's l2: 0.352064
[289]	valid_0's l2: 0.352033
[290]	valid_0's l2: 0.352007
[291]	valid_0's l2: 0.351982
[292]	valid_0's l2: 0.351951
[293]	valid_0's l2: 0.351884
[294]	valid_0's l2: 0.351843
[295]	valid_0's l2: 0.351806
[296]	valid_0's l2: 0.351794
[297]	valid_0's l2: 0.351752
[298]	valid_0's l2: 0.351729
[299]	valid_0's l2: 0.351611
[300]	valid_0's l2: 0.351603
[301]	valid_0's l2: 0.351559
[302]	valid_0's l2: 0.351553
[303]	valid_0's l2: 0.351492
[304]	valid_0's l2: 0.35148
[305]	valid_0's l2: 0.351474
[306]	valid_0's l2: 0.351363
[307]	valid_0's l2: 0.351331
[308]	valid_0's l2: 0.351278
[309]	valid_0's l2: 0.351229
[310]	valid_0's l2: 0.35122
[311]	valid_0's l2: 0.351179
[312]	valid_0's l2: 0.351158
[313]	valid_0's l2: 0.351136
[314]	valid_0's l2: 0.35113
[315]	valid_0's l2: 0.351041
[316]	valid_0's l2: 0.350957
[317]	valid_0's l2: 0.350918
[318]	valid_0's l2: 0.350868
[319]	valid_0's l2: 0.350793
[320]	valid_0's l2: 0.350754
[321]	valid_0's l

[571]	valid_0's l2: 0.343803
[572]	valid_0's l2: 0.343768
[573]	valid_0's l2: 0.343737
[574]	valid_0's l2: 0.343689
[575]	valid_0's l2: 0.343603
[576]	valid_0's l2: 0.343555
[577]	valid_0's l2: 0.343504
[578]	valid_0's l2: 0.343471
[579]	valid_0's l2: 0.343419
[580]	valid_0's l2: 0.343381
[581]	valid_0's l2: 0.343343
[582]	valid_0's l2: 0.343291
[583]	valid_0's l2: 0.343274
[584]	valid_0's l2: 0.343244
[585]	valid_0's l2: 0.343222
[586]	valid_0's l2: 0.343208
[587]	valid_0's l2: 0.343173
[588]	valid_0's l2: 0.343151
[589]	valid_0's l2: 0.343122
[590]	valid_0's l2: 0.343069
[591]	valid_0's l2: 0.343066
[592]	valid_0's l2: 0.343021
[593]	valid_0's l2: 0.342998
[594]	valid_0's l2: 0.342971
[595]	valid_0's l2: 0.342943
[596]	valid_0's l2: 0.342938
[597]	valid_0's l2: 0.342919
[598]	valid_0's l2: 0.342891
[599]	valid_0's l2: 0.342874
[600]	valid_0's l2: 0.342847
[601]	valid_0's l2: 0.34283
[602]	valid_0's l2: 0.342816
[603]	valid_0's l2: 0.342794
[604]	valid_0's l2: 0.342778
[605]	valid_0's

[858]	valid_0's l2: 0.338374
[859]	valid_0's l2: 0.338368
[860]	valid_0's l2: 0.338366
[861]	valid_0's l2: 0.338357
[862]	valid_0's l2: 0.338353
[863]	valid_0's l2: 0.338347
[864]	valid_0's l2: 0.338346
[865]	valid_0's l2: 0.338315
[866]	valid_0's l2: 0.338306
[867]	valid_0's l2: 0.338301
[868]	valid_0's l2: 0.338294
[869]	valid_0's l2: 0.338275
[870]	valid_0's l2: 0.338263
[871]	valid_0's l2: 0.338255
[872]	valid_0's l2: 0.338255
[873]	valid_0's l2: 0.338248
[874]	valid_0's l2: 0.33822
[875]	valid_0's l2: 0.338194
[876]	valid_0's l2: 0.338184
[877]	valid_0's l2: 0.338177
[878]	valid_0's l2: 0.338164
[879]	valid_0's l2: 0.338119
[880]	valid_0's l2: 0.338114
[881]	valid_0's l2: 0.338096
[882]	valid_0's l2: 0.338064
[883]	valid_0's l2: 0.338036
[884]	valid_0's l2: 0.338017
[885]	valid_0's l2: 0.337999
[886]	valid_0's l2: 0.337972
[887]	valid_0's l2: 0.337961
[888]	valid_0's l2: 0.337932
[889]	valid_0's l2: 0.337931
[890]	valid_0's l2: 0.337926
[891]	valid_0's l2: 0.337926
[892]	valid_0's

...

[2246]	valid_0's l2: 0.327174
[2247]	valid_0's l2: 0.327174
[2248]	valid_0's l2: 0.327175
[2249]	valid_0's l2: 0.327159
[2250]	valid_0's l2: 0.327155
[2251]	valid_0's l2: 0.327154
[2252]	valid_0's l2: 0.327152
[2253]	valid_0's l2: 0.327144
[2254]	valid_0's l2: 0.327142
[2255]	valid_0's l2: 0.327142
[2256]	valid_0's l2: 0.327137
[2257]	valid_0's l2: 0.327137
[2258]	valid_0's l2: 0.32713
[2259]	valid_0's l2: 0.327128
[2260]	valid_0's l2: 0.327127
[2261]	valid_0's l2: 0.327109
[2262]	valid_0's l2: 0.32711
[2263]	valid_0's l2: 0.327108
[2264]	valid_0's l2: 0.327105
[2265]	valid_0's l2: 0.327106
[2266]	valid_0's l2: 0.327104
[2267]	valid_0's l2: 0.327101
[2268]	valid_0's l2: 0.3271
[2269]	valid_0's l2: 0.327102
[2270]	valid_0's l2: 0.327103
[2271]	valid_0's l2: 0.327102
[2272]	valid_0's l2: 0.3271
[2273]	valid_0's l2: 0.327098
[2274]	valid_0's l2: 0.327095
[2275]	valid_0's l2: 0.32708
[2276]	valid_0's l2: 0.327079
[2277]	valid_0's l2: 0.327079
[2278]	valid_0's l2: 0.327074
[2279]	valid_0's 

[2526]	valid_0's l2: 0.325908
[2527]	valid_0's l2: 0.325901
[2528]	valid_0's l2: 0.3259
[2529]	valid_0's l2: 0.325898
[2530]	valid_0's l2: 0.325895
[2531]	valid_0's l2: 0.325892
[2532]	valid_0's l2: 0.32589
[2533]	valid_0's l2: 0.325889
[2534]	valid_0's l2: 0.325888
[2535]	valid_0's l2: 0.325885
[2536]	valid_0's l2: 0.325874
[2537]	valid_0's l2: 0.325863
[2538]	valid_0's l2: 0.325856
[2539]	valid_0's l2: 0.325843
[2540]	valid_0's l2: 0.325839
[2541]	valid_0's l2: 0.325836
[2542]	valid_0's l2: 0.325835
[2543]	valid_0's l2: 0.325825
[2544]	valid_0's l2: 0.325812
[2545]	valid_0's l2: 0.325809
[2546]	valid_0's l2: 0.325805
[2547]	valid_0's l2: 0.325804
[2548]	valid_0's l2: 0.3258
[2549]	valid_0's l2: 0.325793
[2550]	valid_0's l2: 0.32579
[2551]	valid_0's l2: 0.325789
[2552]	valid_0's l2: 0.325777
[2553]	valid_0's l2: 0.325764
[2554]	valid_0's l2: 0.325761
[2555]	valid_0's l2: 0.325759
[2556]	valid_0's l2: 0.325748
[2557]	valid_0's l2: 0.325745
[2558]	valid_0's l2: 0.325743
[2559]	valid_0's

[2803]	valid_0's l2: 0.324697
[2804]	valid_0's l2: 0.324664
[2805]	valid_0's l2: 0.324664
[2806]	valid_0's l2: 0.324661
[2807]	valid_0's l2: 0.324657
[2808]	valid_0's l2: 0.324655
[2809]	valid_0's l2: 0.324651
[2810]	valid_0's l2: 0.324645
[2811]	valid_0's l2: 0.324644
[2812]	valid_0's l2: 0.324636
[2813]	valid_0's l2: 0.324629
[2814]	valid_0's l2: 0.324624
[2815]	valid_0's l2: 0.324623
[2816]	valid_0's l2: 0.324625
[2817]	valid_0's l2: 0.324619
[2818]	valid_0's l2: 0.324616
[2819]	valid_0's l2: 0.324611
[2820]	valid_0's l2: 0.324605
[2821]	valid_0's l2: 0.324607
[2822]	valid_0's l2: 0.324608
[2823]	valid_0's l2: 0.324608
[2824]	valid_0's l2: 0.324607
[2825]	valid_0's l2: 0.324607
[2826]	valid_0's l2: 0.324607
[2827]	valid_0's l2: 0.324604
[2828]	valid_0's l2: 0.3246
[2829]	valid_0's l2: 0.324595
[2830]	valid_0's l2: 0.324596
[2831]	valid_0's l2: 0.324596
[2832]	valid_0's l2: 0.324597
[2833]	valid_0's l2: 0.324593
[2834]	valid_0's l2: 0.32459
[2835]	valid_0's l2: 0.324574
[2836]	valid_

[3081]	valid_0's l2: 0.323378
[3082]	valid_0's l2: 0.323361
[3083]	valid_0's l2: 0.323358
[3084]	valid_0's l2: 0.323354
[3085]	valid_0's l2: 0.323331
[3086]	valid_0's l2: 0.32333
[3087]	valid_0's l2: 0.323313
[3088]	valid_0's l2: 0.323301
[3089]	valid_0's l2: 0.323293
[3090]	valid_0's l2: 0.323281
[3091]	valid_0's l2: 0.32328
[3092]	valid_0's l2: 0.32328
[3093]	valid_0's l2: 0.323273
[3094]	valid_0's l2: 0.323272
[3095]	valid_0's l2: 0.323271
[3096]	valid_0's l2: 0.32327
[3097]	valid_0's l2: 0.323267
[3098]	valid_0's l2: 0.323256
[3099]	valid_0's l2: 0.323251
[3100]	valid_0's l2: 0.323246
[3101]	valid_0's l2: 0.323237
[3102]	valid_0's l2: 0.323237
[3103]	valid_0's l2: 0.323234
[3104]	valid_0's l2: 0.323231
[3105]	valid_0's l2: 0.323225
[3106]	valid_0's l2: 0.323222
[3107]	valid_0's l2: 0.32322
[3108]	valid_0's l2: 0.323221
[3109]	valid_0's l2: 0.323221
[3110]	valid_0's l2: 0.323211
[3111]	valid_0's l2: 0.323205
[3112]	valid_0's l2: 0.323189
[3113]	valid_0's l2: 0.323186
[3114]	valid_0'

[3357]	valid_0's l2: 0.322204
[3358]	valid_0's l2: 0.3222
[3359]	valid_0's l2: 0.322196
[3360]	valid_0's l2: 0.322191
[3361]	valid_0's l2: 0.322191
[3362]	valid_0's l2: 0.322192
[3363]	valid_0's l2: 0.322188
[3364]	valid_0's l2: 0.322187
[3365]	valid_0's l2: 0.322186
[3366]	valid_0's l2: 0.32218
[3367]	valid_0's l2: 0.322156
[3368]	valid_0's l2: 0.322155
[3369]	valid_0's l2: 0.322151
[3370]	valid_0's l2: 0.322152
[3371]	valid_0's l2: 0.322149
[3372]	valid_0's l2: 0.322144
[3373]	valid_0's l2: 0.322141
[3374]	valid_0's l2: 0.322138
[3375]	valid_0's l2: 0.322138
[3376]	valid_0's l2: 0.322137
[3377]	valid_0's l2: 0.322137
[3378]	valid_0's l2: 0.322135
[3379]	valid_0's l2: 0.322134
[3380]	valid_0's l2: 0.322129
[3381]	valid_0's l2: 0.322125
[3382]	valid_0's l2: 0.322104
[3383]	valid_0's l2: 0.3221
[3384]	valid_0's l2: 0.322101
[3385]	valid_0's l2: 0.322097
[3386]	valid_0's l2: 0.322092
[3387]	valid_0's l2: 0.322091
[3388]	valid_0's l2: 0.32209
[3389]	valid_0's l2: 0.322091
[3390]	valid_0's

# predict

In [91]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [92]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [93]:

users.loc[:,"user_date"] = pd.to_datetime(users["created_on"])
users.loc[:,"user_year"] = users.loc[:,"user_date"].apply(lambda x:str(x.year))
users.loc[:,"user_month"] = users.loc[:,"user_date"].apply(lambda x:str(x.month))

print(users.shape)
users.head()

(69667, 8)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7


In [94]:

events.loc[:,"event_date"] = pd.to_datetime(events["event_start_at"])
events.loc[:,"event_year"] = events.loc[:,"event_date"].apply(lambda x:str(x.year))
events.loc[:,"event_month"] = events.loc[:,"event_date"].apply(lambda x:str(x.month))
events.loc[:,"event_day"] = events.loc[:,"event_date"].apply(lambda x:str(x.day))
events.loc[:,"event_weekday"] = events.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
events.loc[:,"event_hour"] = events.loc[:,"event_date"].apply(lambda x:str(x.hour))

print(events.shape)
events.head()

(158392, 17)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN,2015-03-01 15:00:00,2015,3,1,6,15
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN,2015-03-07 19:00:00,2015,3,7,5,19
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-14 19:00:00,2015,3,14,5,19


In [95]:
e_pre = prefecture_labeling(events["prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
events = pd.concat((events,e_pre), axis = 1)
print(events.shape)
events.head()

(158392, 18)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18,12
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN,2015-03-01 15:00:00,2015,3,1,6,15,14
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN,2015-03-07 19:00:00,2015,3,7,5,19,14
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18,26
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-14 19:00:00,2015,3,14,5,19,26


In [96]:
u_pre = prefecture_labeling(users["prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(users["gender"])

users = pd.concat((users, u_pre), axis =1)
users = pd.concat((users, gen), axis = 1)
print(users.shape)
users.head()


(69667, 10)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7,46,1
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7,27,1
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7,27,1
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7,8,0


In [97]:
user_event = events
user_event.loc[:,"user_id"] = 1

user_data = pd.merge(events, users, on = "user_id")

print(user_data.shape)
user_data.head()

(158392, 28)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture,user_id,age,gender,prefecture_y,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18,12,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN,2015-03-01 15:00:00,2015,3,1,6,15,14,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN,2015-03-07 19:00:00,2015,3,7,5,19,14,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18,26,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-14 19:00:00,2015,3,14,5,19,26,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0


In [101]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "user_gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "user_gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]


In [102]:


X = user_data[features]
X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
print(X.shape)
X.head()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(158392, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,user_gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,20,99.0,25,44.0,12,1500.0,5000.0,34,0,43,2015,3,8,6,18,2012,6
1,30,99.0,30,49.0,14,1000.0,6500.0,34,0,43,2015,3,1,6,15,2012,6
2,22,99.0,24,39.0,14,1000.0,6000.0,34,0,43,2015,3,7,5,19,2012,6
3,20,99.0,25,44.0,26,1000.0,6000.0,34,0,43,2015,3,8,6,18,2012,6
4,20,99.0,20,39.0,26,1000.0,6000.0,34,0,43,2015,3,14,5,19,2012,6


In [103]:
y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)


In [104]:
y_pred

array([ 2.55169855,  2.00851333,  2.67176253, ...,  2.45252257,
        2.28049733,  2.70704252])

In [105]:
result = pd.DataFrame({"event_id":event_list})
result.loc[:,"predict"] = y_pred
result = result.sort_values(by=["predict"], ascending=False)
result = result.reset_index(drop = True)
result.loc[:,"rank"] = result.index.values + 1

In [106]:
print(result.shape)
result.head()

(158392, 3)


,event_id,predict,rank
0,95079,3.619246,1
1,94220,3.525704,2
2,42368,3.524032,3
3,78590,3.499673,4
4,93770,3.464585,5


In [120]:
def predict_user(user_id):
    
    user_event.loc[:,"user_id"] = user_id

    user_data = pd.merge(events, users, on = "user_id")
    
    X = user_data[features]
    X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
    
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    
    result = pd.DataFrame({"event_id":event_list})
    result.loc[:,"predict"] = y_pred
    result.loc[:,"user_id"] = user_id

    result = result.sort_values(by=["predict"], ascending=False)
    result = result.reset_index(drop = True)
    result.loc[:,"rank"] = result.index.values + 1
    
    return result


In [121]:
start = time.time() #時間計測

predict = predict_user(6)

elapsed_time = time.time() - start #時間計測ここまで

print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


elapsed_time:12.853929281234741[sec]


In [122]:
predict.head(20)

,event_id,predict,user_id,rank
0,157438,3.749938,6,1
1,157448,3.749938,6,2
2,14580,3.711074,6,3
3,157969,3.711074,6,4
4,153442,3.711074,6,5
5,158140,3.710999,6,6
6,151369,3.710999,6,7
7,151353,3.710999,6,8
8,152395,3.710999,6,9
9,151350,3.710999,6,10


In [123]:
user_list = test["user_id"]
event_list = events["event_id"]

In [117]:
print(len(user_list))
print(len(event_list))

2886
158392


In [118]:


k = 20
i = 0
recommends = []

for user in user_list:
    
    start = time.time() #時間計測

    
    i = i +1
    print("user_id:"+str(user))
    print("進捗:"+str(i*100/len(user_list))+"%") 
    
    result = predict_user(user)
    
    recommend_by_user = result.loc[:k-1,:]
    recommends.append(recommend_by_user)
    
    elapsed_time = time.time() - start #時間計測ここまで
    print ("予想残り時間")
    print(elapsed_time * (len(user_list)-i))


user_id:1
進捗:0.03465003465003465%


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


予想残り時間
37139.09034371376
user_id:6
進捗:0.0693000693000693%
予想残り時間
36816.26671695709
user_id:23
進捗:0.10395010395010396%
予想残り時間
35928.07422852516
user_id:43
進捗:0.1386001386001386%


KeyboardInterrupt: 

In [125]:
recommends[1]

,event_id,predict,user_id,rank
0,157438,3.749938,6,1
1,157448,3.749938,6,2
2,14580,3.711074,6,3
3,157969,3.711074,6,4
4,153442,3.711074,6,5
5,158140,3.710999,6,6
6,151369,3.710999,6,7
7,151353,3.710999,6,8
8,152395,3.710999,6,9
9,151350,3.710999,6,10


# 以下工事中

# categorycaなfeatureをダミー化

しなくてよかったっぽい
https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

In [46]:
#categoricalな特徴をダミー化する
#各特徴に専用の関数を作る

def month_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"month"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
def day_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def year_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [2012,2013,2014,2015,2016,2017]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"year"+str(x),cat))
    
    df.columns = column_name
    
    return df
    

def weekday_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"week_day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def hour_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"hour"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
    
    

In [47]:
def gender_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels)).reshape(-1,1))
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df
    

def prefecture_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    label_encoder.classes_
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels))).reshape(-1,1)
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df

categorycalな列を変換

In [48]:
e_pre = prefecture_dummying(X["event_prefercture"])
e_pre.columns = "e_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

KeyError: 'event_prefercture'

In [49]:
u_pre = prefecture_dummying(X["user_prefecture"])
u_pre.columns = "u_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

KeyError: 'user_prefecture'

In [50]:
gender = gender_dummying(X["gender"])
print(gender.shape)
gender.head()

KeyError: 'gender'

In [16]:
u_year = year_dummying(X["user_year"])
u_year.columns = "u_" + u_year.columns.values
print(u_year.shape)
u_year.head()


(1450405, 6)


,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
u_month = month_dummying(X["user_month"])
u_month.columns = "u_" + u_month.columns.values
print(u_month.shape)
u_month.head()

(1450405, 12)


,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
e_year = year_dummying(X["event_year"])
e_year.columns = "e_" + e_year.columns.values
print(e_year.shape)
e_year.head()


(1450405, 6)


,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
e_month = month_dummying(X["event_month"])
e_month.columns = "e_" + e_month.columns.values
print(e_month.shape)
e_month.head()


(1450405, 12)


,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
e_day = day_dummying(X["event_day"])
e_day.columns = "e_" + e_day.columns.values
print(e_day.shape)
e_day.head()

(1450405, 31)


,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
e_weekday = weekday_dummying(X["event_weekday"])
e_weekday.columns = "e_" + e_weekday.columns.values
print(e_weekday.shape)
e_weekday.head()

(1450405, 7)


,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
e_hour = hour_dummying(X["event_hour"])
e_hour.columns = "e_" + e_hour.columns.values
print(e_hour.shape)
e_hour.head()

(1450405, 25)


,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
temp = X[list(set(features) - set(categorical))]
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,gender), axis  =1 )
temp = pd.concat((temp,u_pre), axis = 1)
temp = pd.concat((temp, e_year), axis = 1)
temp = pd.concat((temp, e_month), axis = 1)
temp = pd.concat((temp, e_day), axis = 1)
temp = pd.concat((temp, e_weekday), axis = 1)
temp = pd.concat((temp, e_hour), axis = 1)
temp = pd.concat((temp, u_year), axis = 1)
temp = pd.concat((temp, u_month), axis = 1)
print(temp.shape)
temp.head()

(1450405, 204)


,age,female_price,male_age_upper,female_age_upper,male_age_lower,female_age_lower,male_price,e_その他（海外等）,e_三重県,e_京都府,e_佐賀県,e_兵庫県,e_北海道,e_千葉県,e_和歌山県,e_埼玉県,e_大分県,e_大阪府,e_奈良県,e_宮城県,e_宮崎県,e_富山県,e_山口県,e_山形県,e_山梨県,e_岐阜県,e_岡山県,e_岩手県,e_島根県,e_広島県,e_徳島県,e_愛媛県,e_愛知県,e_新潟県,e_東京都,e_栃木県,e_沖縄県,e_滋賀県,e_熊本県,e_石川県,e_神奈川県,e_福井県,e_福岡県,e_福島県,e_秋田県,e_群馬県,e_茨城県,e_長崎県,e_長野県,e_青森県,e_静岡県,e_香川県,e_高知県,e_鳥取県,e_鹿児島県,女性,男性,u_その他（海外等）,u_三重県,u_京都府,u_佐賀県,u_兵庫県,u_北海道,u_千葉県,u_和歌山県,u_埼玉県,u_大分県,u_大阪府,u_奈良県,u_宮城県,u_宮崎県,u_富山県,u_山口県,u_山形県,u_山梨県,u_岐阜県,u_岡山県,u_岩手県,u_島根県,u_広島県,u_徳島県,u_愛媛県,u_愛知県,u_新潟県,u_東京都,u_栃木県,u_沖縄県,u_滋賀県,u_熊本県,u_石川県,u_神奈川県,u_福井県,u_福岡県,u_福島県,u_秋田県,u_群馬県,u_茨城県,u_長崎県,u_長野県,u_青森県,u_静岡県,u_香川県,u_高知県,u_鳥取県,u_鹿児島県,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,34,4000.0,39.0,39.0,27,27,6500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

# 学習

In [25]:
X = temp

In [26]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1450405, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [28]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1160324,)
float64


In [29]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(290081,)
float64


In [48]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=['c3'])
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [55]:
lgb_train.get_field("gender")

Exception: Cannot get gender before construct Dataset

In [38]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [39]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.552014
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.519943
[3]	valid_0's l2: 0.493896
[4]	valid_0's l2: 0.47265
[5]	valid_0's l2: 0.455379
[6]	valid_0's l2: 0.44132
[7]	valid_0's l2: 0.429646
[8]	valid_0's l2: 0.420306
[9]	valid_0's l2: 0.412629
[10]	valid_0's l2: 0.406207
[11]	valid_0's l2: 0.401102
[12]	valid_0's l2: 0.39677
[13]	valid_0's l2: 0.393205
[14]	valid_0's l2: 0.390245
[15]	valid_0's l2: 0.387918
[16]	valid_0's l2: 0.385837
[17]	valid_0's l2: 0.384236
[18]	valid_0's l2: 0.382449
[19]	valid_0's l2: 0.381205
[20]	valid_0's l2: 0.379905
[21]	valid_0's l2: 0.379007
[22]	valid_0's l2: 0.377982
[23]	valid_0's l2: 0.377311
[24]	valid_0's l2: 0.376618
[25]	valid_0's l2: 0.376054
[26]	valid_0's l2: 0.375384
[27]	valid_0's l2: 0.374938
[28]	valid_0's l2: 0.374442
[29]	valid_0's l2: 0.374046
[30]	valid_0's l2: 0.373613
[31]	valid_0's l2: 0.373235
[32]	valid_0's l2: 0.372919
[33]	valid_0's l2: 0.372652
[34]	valid_0's l2: 0.37239

[292]	valid_0's l2: 0.357452
[293]	valid_0's l2: 0.357413
[294]	valid_0's l2: 0.357363
[295]	valid_0's l2: 0.357342
[296]	valid_0's l2: 0.357293
[297]	valid_0's l2: 0.357288
[298]	valid_0's l2: 0.357281
[299]	valid_0's l2: 0.357277
[300]	valid_0's l2: 0.35726
[301]	valid_0's l2: 0.357242
[302]	valid_0's l2: 0.357225
[303]	valid_0's l2: 0.357157
[304]	valid_0's l2: 0.357143
[305]	valid_0's l2: 0.35712
[306]	valid_0's l2: 0.357103
[307]	valid_0's l2: 0.357089
[308]	valid_0's l2: 0.357066
[309]	valid_0's l2: 0.357062
[310]	valid_0's l2: 0.357054
[311]	valid_0's l2: 0.357045
[312]	valid_0's l2: 0.357041
[313]	valid_0's l2: 0.357033
[314]	valid_0's l2: 0.35699
[315]	valid_0's l2: 0.356975
[316]	valid_0's l2: 0.356938
[317]	valid_0's l2: 0.356929
[318]	valid_0's l2: 0.356916
[319]	valid_0's l2: 0.356906
[320]	valid_0's l2: 0.356866
[321]	valid_0's l2: 0.356829
[322]	valid_0's l2: 0.356827
[323]	valid_0's l2: 0.356798
[324]	valid_0's l2: 0.356779
[325]	valid_0's l2: 0.356751
[326]	valid_0's l

[576]	valid_0's l2: 0.351405
[577]	valid_0's l2: 0.351368
[578]	valid_0's l2: 0.351359
[579]	valid_0's l2: 0.351353
[580]	valid_0's l2: 0.351336
[581]	valid_0's l2: 0.351325
[582]	valid_0's l2: 0.351304
[583]	valid_0's l2: 0.3513
[584]	valid_0's l2: 0.351295
[585]	valid_0's l2: 0.351286
[586]	valid_0's l2: 0.351286
[587]	valid_0's l2: 0.351284
[588]	valid_0's l2: 0.351278
[589]	valid_0's l2: 0.351271
[590]	valid_0's l2: 0.351267
[591]	valid_0's l2: 0.351224
[592]	valid_0's l2: 0.351215
[593]	valid_0's l2: 0.351204
[594]	valid_0's l2: 0.351199
[595]	valid_0's l2: 0.351192
[596]	valid_0's l2: 0.351185
[597]	valid_0's l2: 0.351174
[598]	valid_0's l2: 0.35116
[599]	valid_0's l2: 0.351155
[600]	valid_0's l2: 0.351155
[601]	valid_0's l2: 0.351131
[602]	valid_0's l2: 0.351131
[603]	valid_0's l2: 0.351103
[604]	valid_0's l2: 0.351097
[605]	valid_0's l2: 0.351073
[606]	valid_0's l2: 0.351051
[607]	valid_0's l2: 0.351028
[608]	valid_0's l2: 0.351026
[609]	valid_0's l2: 0.351023
[610]	valid_0's l

[861]	valid_0's l2: 0.346313
[862]	valid_0's l2: 0.346316
[863]	valid_0's l2: 0.346314
[864]	valid_0's l2: 0.34631
[865]	valid_0's l2: 0.346285
[866]	valid_0's l2: 0.346282
[867]	valid_0's l2: 0.346243
[868]	valid_0's l2: 0.346226
[869]	valid_0's l2: 0.346224
[870]	valid_0's l2: 0.346221
[871]	valid_0's l2: 0.346218
[872]	valid_0's l2: 0.346217
[873]	valid_0's l2: 0.346215
[874]	valid_0's l2: 0.346203
[875]	valid_0's l2: 0.346204
[876]	valid_0's l2: 0.346183
[877]	valid_0's l2: 0.346157
[878]	valid_0's l2: 0.346153
[879]	valid_0's l2: 0.346133
[880]	valid_0's l2: 0.346097
[881]	valid_0's l2: 0.346073
[882]	valid_0's l2: 0.346048
[883]	valid_0's l2: 0.346025
[884]	valid_0's l2: 0.346005
[885]	valid_0's l2: 0.345979
[886]	valid_0's l2: 0.345948
[887]	valid_0's l2: 0.345929
[888]	valid_0's l2: 0.345928
[889]	valid_0's l2: 0.345926
[890]	valid_0's l2: 0.345874
[891]	valid_0's l2: 0.345875
[892]	valid_0's l2: 0.345871
[893]	valid_0's l2: 0.345867
[894]	valid_0's l2: 0.345864
[895]	valid_0's

[1142]	valid_0's l2: 0.342589
[1143]	valid_0's l2: 0.342564
[1144]	valid_0's l2: 0.342552
[1145]	valid_0's l2: 0.342528
[1146]	valid_0's l2: 0.342507
[1147]	valid_0's l2: 0.342501
[1148]	valid_0's l2: 0.342485
[1149]	valid_0's l2: 0.342483
[1150]	valid_0's l2: 0.342475
[1151]	valid_0's l2: 0.342462
[1152]	valid_0's l2: 0.342451
[1153]	valid_0's l2: 0.34245
[1154]	valid_0's l2: 0.342443
[1155]	valid_0's l2: 0.342424
[1156]	valid_0's l2: 0.342403
[1157]	valid_0's l2: 0.342404
[1158]	valid_0's l2: 0.342402
[1159]	valid_0's l2: 0.342383
[1160]	valid_0's l2: 0.342364
[1161]	valid_0's l2: 0.342364
[1162]	valid_0's l2: 0.34236
[1163]	valid_0's l2: 0.342353
[1164]	valid_0's l2: 0.342347
[1165]	valid_0's l2: 0.342344
[1166]	valid_0's l2: 0.34234
[1167]	valid_0's l2: 0.34233
[1168]	valid_0's l2: 0.342326
[1169]	valid_0's l2: 0.342314
[1170]	valid_0's l2: 0.34228
[1171]	valid_0's l2: 0.342267
[1172]	valid_0's l2: 0.342265
[1173]	valid_0's l2: 0.342249
[1174]	valid_0's l2: 0.342249
[1175]	valid_0'

[1419]	valid_0's l2: 0.339603
[1420]	valid_0's l2: 0.339598
[1421]	valid_0's l2: 0.339586
[1422]	valid_0's l2: 0.339585
[1423]	valid_0's l2: 0.339581
[1424]	valid_0's l2: 0.339567
[1425]	valid_0's l2: 0.339552
[1426]	valid_0's l2: 0.339548
[1427]	valid_0's l2: 0.339537
[1428]	valid_0's l2: 0.339506
[1429]	valid_0's l2: 0.339505
[1430]	valid_0's l2: 0.339502
[1431]	valid_0's l2: 0.339492
[1432]	valid_0's l2: 0.339483
[1433]	valid_0's l2: 0.339472
[1434]	valid_0's l2: 0.339467
[1435]	valid_0's l2: 0.339468
[1436]	valid_0's l2: 0.339461
[1437]	valid_0's l2: 0.339418
[1438]	valid_0's l2: 0.339408
[1439]	valid_0's l2: 0.339409
[1440]	valid_0's l2: 0.339405
[1441]	valid_0's l2: 0.339405
[1442]	valid_0's l2: 0.339404
[1443]	valid_0's l2: 0.339396
[1444]	valid_0's l2: 0.339374
[1445]	valid_0's l2: 0.339368
[1446]	valid_0's l2: 0.339364
[1447]	valid_0's l2: 0.339357
[1448]	valid_0's l2: 0.339337
[1449]	valid_0's l2: 0.339329
[1450]	valid_0's l2: 0.339325
[1451]	valid_0's l2: 0.339323
[1452]	val

[1694]	valid_0's l2: 0.337502
[1695]	valid_0's l2: 0.337501
[1696]	valid_0's l2: 0.337492
[1697]	valid_0's l2: 0.337488
[1698]	valid_0's l2: 0.337482
[1699]	valid_0's l2: 0.337482
[1700]	valid_0's l2: 0.337477
[1701]	valid_0's l2: 0.337474
[1702]	valid_0's l2: 0.337467
[1703]	valid_0's l2: 0.337456
[1704]	valid_0's l2: 0.337421
[1705]	valid_0's l2: 0.337421
[1706]	valid_0's l2: 0.337415
[1707]	valid_0's l2: 0.337415
[1708]	valid_0's l2: 0.337413
[1709]	valid_0's l2: 0.337411
[1710]	valid_0's l2: 0.337409
[1711]	valid_0's l2: 0.337402
[1712]	valid_0's l2: 0.337402
[1713]	valid_0's l2: 0.337402
[1714]	valid_0's l2: 0.337393
[1715]	valid_0's l2: 0.337383
[1716]	valid_0's l2: 0.33738
[1717]	valid_0's l2: 0.337378
[1718]	valid_0's l2: 0.337378
[1719]	valid_0's l2: 0.337374
[1720]	valid_0's l2: 0.337364
[1721]	valid_0's l2: 0.337344
[1722]	valid_0's l2: 0.337341
[1723]	valid_0's l2: 0.337315
[1724]	valid_0's l2: 0.337311
[1725]	valid_0's l2: 0.337305
[1726]	valid_0's l2: 0.337285
[1727]	vali

In [41]:
y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

In [42]:
y_pred

array([ 2.31565148,  2.47586269,  2.37553958, ...,  1.19048797,
        1.33693581,  1.00767751])

# predict

In [51]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")

In [52]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [53]:
user_list = test["user_id"]
event_list = events["event_id"]

In [58]:
events.loc[:,"user_id"] = 1
user_data = pd.merge(events, users, on = "user_id")
data = data.rename(columns = {"prefecture_x": "event_prefercture", "prefecture_y":"user_prefecture"})

data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [32]:
X = data[features]
print(X.shape)
X.head()

(1442610, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6


In [33]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(X["gender"])

temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()

(1442610, 14)
(1442610, 17)


,male_age_lower,user_month,user_year,male_price,female_age_upper,female_age_lower,event_year,female_price,event_month,age,male_age_upper,event_day,event_weekday,event_hour,gender,event_prefecture,user_prefecture
0,27,6,2012,6500.0,39.0,27,2015,4000.0,3,34,39.0,22,6,14,0,27,43
1,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
2,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
3,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
4,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
